# Travel Planner Exercise

In this exercise, we'll create a travel planner that:
1. Takes trip requirements
2. Creates detailed itineraries
3. Stores travel tips in memory

The focus is on crafting and experimenting with prompts!

In [ ]:
import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)

sys.path.append(grandparent_dir)

### 1. Setup

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore
from semantic_kernel.planners import SequentialPlanner

kernel = Kernel()

# Chat service
service_id = "chat-gpt"
chat_service = AzureChatCompletion(
    service_id=service_id,
)
kernel.add_service(chat_service)

# Embedding service
embedding_service_id = "embeddings"
kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))

# Memory setup
memory_store = VolatileMemoryStore()
memory = SemanticTextMemory(storage=memory_store, embeddings_generator=kernel.get_service(embedding_service_id))

# Planner setup
planner = SequentialPlanner(kernel)

### 2. Define Base Prompts (Experiment with these!)

These prompts work together to plan trips. Try modifying them to see how it changes the results!

In [ ]:
# Prompt 1: Create initial travel plan
# Try changing the format, adding constraints, or modifying the style!
create_plan_prompt = """
Trip Requirements:
Destination: {{$destination}}
Duration: {{$duration}}
Interests: {{$interests}}
Budget Level: {{$budget}}

Create a day-by-day travel itinerary that:
1. Maximizes time efficiently
2. Groups nearby activities together
3. Includes meal suggestions
4. Stays within budget level

Format the itinerary as:
🗓️ Day [number]: [theme for the day]
- Morning: [activities]
- Afternoon: [activities]
- Evening: [activities]
- 🍽️ Food spots: [suggestions]
- 💰 Expected costs: [cost range]
"""

# Prompt 2: Generate travel tips
# Try adding different categories or changing the focus!
travel_tips_prompt = """
Destination: {{$destination}}
Season: {{$season}}

Generate practical travel tips for this destination.
Focus on local insights and common tourist mistakes to avoid.

Format tips as:
🎒 Packing Tips:
- [tips]

🚶‍♂️ Getting Around:
- [tips]

💡 Local Customs:
- [tips]

⚠️ Tourist Traps to Avoid:
- [tips]
"""

# Prompt 3: Optimize itinerary
# Try adding different optimization criteria!
optimize_prompt = """
Original Itinerary: {{$itinerary}}
Weather Forecast: {{$weather}}

Optimize this itinerary based on the weather forecast.
Suggest indoor alternatives for rainy periods.
Adjust timing for temperature peaks.

Format changes as:
📅 Original Plan: [activity]
🔄 Suggested Change: [new plan]
❓ Reason: [explanation]
"""

# Create semantic functions
create_plan = kernel.add_function(
    prompt=create_plan_prompt,
    function_name="CreatePlan",
    plugin_name="TravelPlanner",
)

generate_tips = kernel.add_function(
    prompt=travel_tips_prompt,
    function_name="GenerateTips",
    plugin_name="TravelPlanner",
)

optimize_plan = kernel.add_function(
    prompt=optimize_prompt,
    function_name="OptimizePlan",
    plugin_name="TravelPlanner",
)

### 3. Example Usage

Try these examples, then modify the prompts above to see how it changes the results!

In [ ]:
# Example 1: Create a travel plan
trip_details = {
    "destination": "Kyoto, Japan",
    "duration": "3 days",
    "interests": "temples, traditional culture, food",
    "budget": "medium"
}

plan = await kernel.invoke(
    create_plan,
    **trip_details
)
print("Initial Plan:\n", plan)

# Example 2: Get travel tips
tips = await kernel.invoke(
    generate_tips,
    destination=trip_details["destination"],
    season="autumn"
)
print("\nTravel Tips:\n", tips)

# Example 3: Optimize for weather
weather = "Day 1: Rain in morning, clear afternoon\nDay 2: Sunny all day\nDay 3: Light rain all day"
optimized = await kernel.invoke(
    optimize_plan,
    itinerary=str(plan),
    weather=weather
)
print("\nOptimized Plan:\n", optimized)

# Store tips in memory
await memory.save_information(
    collection="travel_tips",
    id=f"tips_{trip_details['destination'].lower().replace(' ', '_')}",
    text=str(tips)
)

### 4. Search Travel Tips

Try searching for tips you've stored!

In [ ]:
# Search for relevant tips
search_terms = ["cultural etiquette", "transportation", "food recommendations"]

for term in search_terms:
    print(f"\nSearching tips related to '{term}':\n")
    results = await memory.search("travel_tips", term)
    for result in results:
        print(f"Match (relevance: {result.relevance}):\n{result.text}\n")

### 5. Additional exercises

Try these exercises to experiment with prompt engineering:

1. Modify the create_plan_prompt to:
   - Add sustainability ratings for activities
   - Include accessibility information
   - Suggest photo opportunities

2. Enhance the travel_tips_prompt with:
   - Safety recommendations
   - Language essentials
   - Social media spots

3. Expand the optimize_prompt to consider:
   - Local events and festivals
   - Peak tourist times
   - Public holiday impacts

4. Create a new prompt that:
   - Suggests souvenir shopping locations
   - Plans activities for specific age groups
   - Includes local transportation options

5. Advanced Challenge:
   - Create a prompt that combines multiple plans into a group itinerary
   - Add budget tracking across activities
   - Include contingency plans for each day

Remember: The focus is on prompt engineering. Try different formats, styles, and requirements to see how they affect the results!